In [1]:
# import the necessary modules

In [2]:
import pandas as pd
import numpy as np
import os
import datetime

In [3]:
# set the home directory. it will store our data.

In [4]:
root_path = os.path.expanduser('~')

In [5]:
# we read data from csv and parquet into dataframes.

In [6]:
carriers = pd.read_csv(os.path.join(root_path, 'source', 'carriers.csv'))
airports = pd.read_csv(os.path.join(root_path, 'source', 'airports.csv'), keep_default_na=False)
flights = pd.read_csv(os.path.join(root_path, 'source', 'flights.csv'))
carriers_parquet = pd.read_parquet(os.path.join(root_path, 'raw', 'carriers'))

In [7]:
# set the required variables

In [8]:
carriers.columns = carriers.columns.str.title()
carriers_parquet.columns = carriers_parquet.columns.str.title()
result_list = []
column_result_list = ['Table', 'DQ_check', 'Column', 'Status', 'Bad_Data']

In [9]:
# Let's create a function that will compare two dataframes.
# The output will be the number of lines that differ in the dataframes.
# If the result is 0, then there is no difference.

In [10]:
def compare_two_dataframe(df1, df2):
    """
    return:
        0 - test passed
        != 0 - test failed
    """
    df_diff = df1.merge(df2,indicator=True, how='left').loc[lambda x : x['_merge']!='both']
    return df_diff['_merge'].count()

In [11]:
# We create a function that will compare two parameters dependent on each other.
# The output will be the number of differences.
# If the result is 0, then there is no difference.

In [12]:
def chech_parameter_dependence(df, column1, condition1, column2, condition2):
    """
    return:
        0 - test passed
        != 0 - test failed
    """
    return df[(df[column1] == condition1) & (df[column2] == condition2)].shape[0]

In [13]:
# We create a function that will compare two parameters dependent on each other. The second parameter is NOT NULL.
# The output will be the number of differences.
# If the result is 0, then there is no difference.

In [14]:
def chech_parameter_and_null(df, column1, condition1, column2):
    """
    return:
        0 - test passed
        != 0 - test failed
    """
    return df[(df[column1] == condition1) & (df[column2].notnull())].shape[0]

In [15]:
# We create a function that will compare two parameters dependent on each other. The condition for the second parameter is inverted.
# The output will be the number of differences.
# If the result is 0, then there is no difference.

In [16]:
def chech_parameter_and_not_condition2(df, column1, condition1, column2, condition2):
    """
    return:
        0 - test passed
        != 0 - test failed
    """
    return df[(df[column1] == condition1) & (~df[column2].isin(condition2))].shape[0]

In [17]:
# We create a function that will parse a text line containing time. For example "1525".
# In the output, this string is converted to the number of minutes.

In [18]:
def string_to_minutes(df_work, column):
    df_work[column] = df_work[column].str[-4:-2].astype(int) * 60 + df_work[column].str[-2:].astype(int)
    return df_work

In [19]:
# We create a function that will calculate the difference in values in two columns and compare with the third.
# If the result differs, then 1 will be written to the 'Result' column, otherwise 0.
# The output will be the sum of the 'Result' column.
# # If the result is 0, then there is no difference.

In [20]:
def difference_two_column_use_conditional_column(df_work, column_condition, column1, column2):
    """
    return:
        0 - test passed
        != 0 - test failed
    """
    df_work['Result'] = np.where(df_work[column_condition] == df_work[column1] - df_work[column2], 0, 1)
    return df_work['Result'].sum()

In [21]:
# check if data is loaded from files

In [22]:
carriers.head()

,Code,Description
0,02Q,Titan Airways
1,04Q,Tradewind Aviation
2,05Q,"Comlux Aviation, AG"
3,06Q,Master Top Linhas Aereas Ltd.
4,07Q,Flair Airlines Ltd.


In [23]:
airports.head()

,iata,airport,city,state,country,lat,long
0,00M,Thigpen,Bay Springs,MS,USA,31.95376472,-89.23450472
1,00R,Livingston Municipal,Livingston,TX,USA,30.68586111,-95.01792778
2,00V,Meadow Lake,Colorado Springs,CO,USA,38.94574889,-104.5698933
3,01G,Perry-Warsaw,Perry,NY,USA,42.74134667,-78.05208056
4,01J,Hilliard Airpark,Hilliard,FL,USA,30.6880125,-81.90594389


In [24]:
flights.head()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2008.0,5.0,1.0,4.0,859.0,905.0,1025.0,1030.0,AA,1755.0,...,5.0,10.0,0.0,NaN,0.0,NaN,NaN,8.0,NaN,NaN
1,2008.0,5.0,1.0,4.0,859.0,905.0,1025.0,1030.0,AA,1755.0,...,5.0,10.0,0.0,NaN,0.0,NaN,5.0,NaN,NaN,NaN
2,2008.0,5.0,1.0,4.0,859.0,905.0,1025.0,1030.0,AA,1856.0,...,5.0,10.0,0.0,NaN,0.0,3.0,NaN,NaN,NaN,NaN
3,2008.0,5.0,2.0,5.0,901.0,905.0,1104.0,1030.0,AA,1755.0,...,7.0,35.0,0.0,NaN,0.0,0.0,0.0,34.0,0.0,0.0
4,2008.0,5.0,3.0,6.0,901.0,905.0,1034.0,1030.0,AA,1755.0,...,5.0,17.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN


In [25]:
carriers_parquet.head()

,Code,Description
0,02Q,Titan Airways
1,04Q,Tradewind Aviation
2,05Q,"Comlux Aviation, AG"
3,06Q,Master Top Linhas Aereas Ltd.
4,07Q,Flair Airlines Ltd.


In [26]:
# Carrier: Completeness by records
# the test result will be written to the result_list variable.
# compare data in two dataframes in the 'Code' and 'Description' columns

In [27]:
result = compare_two_dataframe(carriers, carriers_parquet)
result_list.append(['Carriers',
                   'Completeness',
                   'Code, Description',
                   True if result == 0 else False,
                   'No bad data.' if result == 0 else f'Found {result} differences between data in source and raw.'])

In [28]:
# Airports: Consistency check for state
# 'NA' meaning no state specified can be applied for non US countries only. For US it should be a valid state. 

In [29]:
result = chech_parameter_dependence(airports, 'country', 'USA', 'state', 'NA')
result_list.append(['Airports',
                   'Consistency',
                   'country, state',
                   True if result == 0 else False,
                   'No bad data.' if result == 0 else f'Found {result} inconsistencies between the data in the columns.'])

In [30]:
# Flights: Consistency for CancellationCode
# NULL if Cancelled = 0
# A/B/C if Cancelled = 1

In [31]:
result = chech_parameter_and_null(flights, 'Cancelled', 0, 'CancellationCode')
result_list.append(['Flights',
                   'Consistency',
                   'Cancelled, CancellationCode',
                   True if result == 0 else False,
                   'No bad data.' if result == 0 else f'Found {result} inconsistencies between the data in the columns. If the value is Canceled = 0'])

result = chech_parameter_and_not_condition2(flights, 'Cancelled',  1, 'CancellationCode', ['A', 'B', 'C'])
result_list.append(['Flights',
                   'Consistency',
                   'Cancelled, CancellationCode',
                   True if result == 0 else False,
                   'No bad data.' if result == 0 else f'Found {result} inconsistencies between the data in the columns. If the value is Canceled = 1'])


In [32]:
# Flights: Consistency check for CRSElapsedTime
# Calculated column: CRSArrTime - CRSDepTime (mins)
# Explanation. Since the task did not have a test for checking data in the CRSArrTime and CRSDepTime columns,
# we assume that the data in them does not contain errors and NULL.
# Then we convert the data in columns to minutes and compare the difference with CRSElapsedTime.

In [33]:
df = '0000' + flights.loc[(flights['CRSElapsedTime'].notnull()) & (flights['CRSArrTime'].notnull()) & (flights['CRSDepTime'].notnull()),
['CRSElapsedTime', 'CRSArrTime', 'CRSDepTime']].astype(int).astype(str)
df['CRSElapsedTime'] = df['CRSElapsedTime'].astype(int)
df = string_to_minutes(df, 'CRSArrTime')
df = string_to_minutes(df, 'CRSDepTime')
result = difference_two_column_use_conditional_column(df, 'CRSElapsedTime', 'CRSArrTime', 'CRSDepTime')
result_list.append(['Flights',
                   'Consistency',
                   'CRSElapsedTime, CRSArrTime, CRSDepTime',
                   True if result == 0 else False,
                   'No bad data.' if result == 0 else f'Found {result} inconsistencies between the data in the columns. (CRSElapsedTime <> CRSArrTime - CRSDepTime)'])


In [34]:
# We convert the result to a dataframe and display it.

In [35]:
pd.set_option('display.max_colwidth', None)
df_result = pd.DataFrame (result_list, columns = column_result_list)
df_result

,Table,DQ_check,Column,Status,Bad_Data
0,Carriers,Completeness,"Code, Description",False,Found 7 differences between data in source and raw.
1,Airports,Consistency,"country, state",False,Found 8 inconsistencies between the data in the columns.
2,Flights,Consistency,"Cancelled, CancellationCode",False,Found 3 inconsistencies between the data in the columns. If the value is Canceled = 0
3,Flights,Consistency,"Cancelled, CancellationCode",False,Found 1 inconsistencies between the data in the columns. If the value is Canceled = 1
4,Flights,Consistency,"CRSElapsedTime, CRSArrTime, CRSDepTime",False,Found 244 inconsistencies between the data in the columns. (CRSElapsedTime <> CRSArrTime - CRSDepTime)


In [36]:
# write .xlsx and .csv

In [37]:
df_result.to_excel(os.path.join(root_path, 'test_result.xlsx'),
             sheet_name='Test result')
df_result.to_csv(os.path.join(root_path, 'test_result.csv'))